In [1]:
!pip install -q miditok symusic pretty_midi librosa transformers torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.0/159.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.9 MB/s eta 0:00:00
   

In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, classification_report, confusion_matrix
from tqdm.notebook import tqdm

# Audio & CLAP
import librosa
from transformers import ClapProcessor, ClapModel

# Text & BERT
from transformers import AutoTokenizer, AutoModel

# MIDI
from miditok import REMI, TokenizerConfig
from symusic import Score


2025-12-15 06:22:23.169801: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765779743.355275      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765779743.411196      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
BASE = Path("/kaggle/input/multimodal/archive/archive")
DATASET_DIR = BASE / "dataset"

AUDIO_DIR  = DATASET_DIR / "Audio"
LYRICS_DIR = DATASET_DIR / "Lyrics"
MIDI_DIR   = DATASET_DIR / "MIDIs"

CSV_PATH   = DATASET_DIR / "dataset info.csv"
CAT_PATH   = DATASET_DIR / "categories.txt"
CLUST_PATH = DATASET_DIR / "clusters.txt"

# Load labels
df = pd.read_csv(CSV_PATH, encoding="utf-16", sep=";", engine="python", on_bad_lines="skip")

with open(CAT_PATH) as f:
    categories = [line.strip() for line in f.readlines()]

with open(CLUST_PATH) as f:
    clusters = [line.strip() for line in f.readlines()]

track_ids = sorted([f.split(".")[0] for f in os.listdir(AUDIO_DIR)])

df_labels = pd.DataFrame({
    "track_id": track_ids,
    "category": categories,
    "cluster": clusters
})
df_labels["cluster_id"] = df_labels["cluster"].apply(lambda x: int(x.split(" ")[1]))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

🚀 Using device: cuda


In [4]:
print("\n📦 Loading pretrained models...")

# --- CLAP (AUDIO) ---
processor = ClapProcessor.from_pretrained("laion/clap-htsat-unfused")
clap = ClapModel.from_pretrained("laion/clap-htsat-unfused").to(device)
clap.eval()
for param in clap.parameters():
    param.requires_grad = False

# --- BERT (TEXT) ---
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert = AutoModel.from_pretrained("bert-base-uncased").to(device)
bert.eval()
for param in bert.parameters():
    param.requires_grad = False

# --- MIDI TOKENIZER ---
cfg = TokenizerConfig(
    pitch_range=(21, 109),
    beat_res={(0, 4): 8, (4, 12): 4},
    num_velocities=32,
    use_chords=True,
    use_tempos=True
)
tok = REMI(cfg)



📦 Loading pretrained models...


preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
def encode_audio(path, sr=48000):
    try:
        y, _ = librosa.load(path, sr=sr, mono=True)
        inputs = processor(audios=y, sampling_rate=sr, return_tensors="pt").to(device)
        with torch.no_grad():
            emb = clap.get_audio_features(input_features=inputs["input_features"])
        return emb.cpu().numpy().squeeze()
    except Exception as e:
        return np.zeros(512)

def clean_text(t):
    t = t.lower()
    t = re.sub(r"[^a-z0-9\s]", " ", t)
    return re.sub(r"\s+", " ", t).strip()

def encode_text(tid):
    lyr_path = LYRICS_DIR / f"{tid}.txt"
    if lyr_path.exists():
        text = clean_text(open(lyr_path, encoding="utf-8", errors="ignore").read())
    else:
        text = ""
    enc = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=256).to(device)
    with torch.no_grad():
        emb = bert(**enc).last_hidden_state.mean(dim=1)
    return emb.cpu().numpy().squeeze()


In [6]:
class MidiEncoder(nn.Module):
    def __init__(self, vocab, d=256):
        super().__init__()
        self.pad = vocab["PAD_None"]
        self.out_dim = d
        self.emb = nn.Embedding(len(vocab), d, padding_idx=self.pad)
        layer = nn.TransformerEncoderLayer(d_model=d, nhead=8, batch_first=True)
        self.enc = nn.TransformerEncoder(layer, num_layers=2)
    
    def forward(self, ids):
        mask = ids == self.pad
        x = self.emb(ids)
        x = self.enc(x, src_key_padding_mask=mask)
        return x.mean(dim=1)

In [7]:
print("\n🔄 Extracting features...")

# Audio
print("  → Audio embeddings...")
X_audio = np.vstack([encode_audio(AUDIO_DIR / f"{tid}.mp3") for tid in tqdm(track_ids, desc="Audio")])
np.save("/kaggle/working/X_audio.npy", X_audio)

# Text
print("  → Text embeddings...")
X_text = np.vstack([encode_text(tid) for tid in tqdm(track_ids, desc="Text")])
np.save("/kaggle/working/X_text.npy", X_text)

# MIDI
print("  → MIDI embeddings...")
midi_encoder = MidiEncoder(tok.vocab).to(device)
midi_encoder.eval()

X_midi = []
for tid in tqdm(track_ids, desc="MIDI"):
    path = MIDI_DIR / f"{tid}.mid"
    if not path.exists():
        X_midi.append(np.zeros(256))
        continue
    try:
        score = Score(str(path))
        seq = tok(score).ids
        seq = seq[:1024] + [midi_encoder.pad] * (1024 - len(seq))
        ids = torch.tensor([seq]).to(device)
        with torch.no_grad():
            emb = midi_encoder(ids).cpu().numpy().squeeze()
        X_midi.append(emb)
    except:
        X_midi.append(np.zeros(256))

X_midi = np.vstack(X_midi)

# Handle missing MIDI data
valid_mask = (X_midi.sum(axis=1) != 0)
if valid_mask.any():
    midi_mean = X_midi[valid_mask].mean(axis=0)
    for i in range(len(X_midi)):
        if not valid_mask[i]:
            X_midi[i] = midi_mean

np.save("/kaggle/working/X_midi.npy", X_midi)

print(f"\n✅ Features extracted:")
print(f"   Audio: {X_audio.shape}")
print(f"   Text:  {X_text.shape}")
print(f"   MIDI:  {X_midi.shape}")


🔄 Extracting features...
  → Audio embeddings...


Audio:   0%|          | 0/903 [00:00<?, ?it/s]

  → Text embeddings...


Text:   0%|          | 0/903 [00:00<?, ?it/s]

  → MIDI embeddings...


MIDI:   0%|          | 0/903 [00:00<?, ?it/s]


✅ Features extracted:
   Audio: (903, 512)
   Text:  (903, 768)
   MIDI:  (903, 256)


In [8]:
emotion_ids = df_labels["cluster_id"].values

train_idx, val_idx = train_test_split(
    np.arange(len(emotion_ids)),
    test_size=0.2,
    stratify=emotion_ids,
    random_state=42
)

Xa_tr, Xa_val = X_audio[train_idx], X_audio[val_idx]
Xt_tr, Xt_val = X_text[train_idx], X_text[val_idx]
Xm_tr, Xm_val = X_midi[train_idx], X_midi[val_idx]
y_tr, y_val = emotion_ids[train_idx], emotion_ids[val_idx]

print(f"\n📊 Dataset split:")
print(f"   Train: {len(train_idx)} samples")
print(f"   Val:   {len(val_idx)} samples")



📊 Dataset split:
   Train: 722 samples
   Val:   181 samples


In [9]:
class MDataset(Dataset):
    def __init__(self, A, T, M, y):
        self.A = torch.tensor(A).float()
        self.T = torch.tensor(T).float()
        self.M = torch.tensor(M).float()
        self.y = torch.tensor(y - 1).long()  # Convert to 0-indexed
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.A[idx], self.T[idx], self.M[idx], self.y[idx]

train_loader = DataLoader(MDataset(Xa_tr, Xt_tr, Xm_tr, y_tr), batch_size=32, shuffle=True)
val_loader = DataLoader(MDataset(Xa_val, Xt_val, Xm_val, y_val), batch_size=32)

In [10]:
class CrossAttention(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.attn = nn.MultiheadAttention(d, 4, batch_first=True)
        self.norm = nn.LayerNorm(d)
    
    def forward(self, q, kv):
        out, _ = self.attn(q, kv, kv)
        return self.norm(q + out)

class Gating(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.Wa = nn.Linear(d, 1)
        self.Wt = nn.Linear(d, 1)
        self.Wm = nn.Linear(d, 1)
    
    def forward(self, a, t, m):
        wa = self.Wa(a)
        wt = self.Wt(t)
        wm = self.Wm(m)
        w = torch.softmax(torch.cat([wa, wt, wm], dim=1), dim=1)
        return w[:, 0:1] * a + w[:, 1:2] * t + w[:, 2:3] * m

class MultiModalFusion(nn.Module):
    def __init__(self, d=256):
        super().__init__()
        self.pA = nn.Sequential(nn.Linear(512, d), nn.ReLU(), nn.LayerNorm(d))
        self.pT = nn.Sequential(nn.Linear(768, d), nn.ReLU(), nn.LayerNorm(d))
        self.pM = nn.Sequential(nn.Linear(256, d), nn.ReLU(), nn.LayerNorm(d))
        
        self.ca_AT = CrossAttention(d)
        self.ca_TM = CrossAttention(d)
        self.ca_MA = CrossAttention(d)
        
        self.gate = Gating(d)
        self.dropout_fuse = nn.Dropout(0.3)
        
        self.clf = nn.Sequential(
            nn.Linear(d, 512), nn.ReLU(), nn.Dropout(0.4),
            nn.Linear(512, 128), nn.ReLU(), nn.Dropout(0.4),
            nn.Linear(128, 5)
        )
    
    def forward(self, A, T, M):
        A = self.pA(A).unsqueeze(1)
        T = self.pT(T).unsqueeze(1)
        M = self.pM(M).unsqueeze(1)
        
        A2 = self.ca_AT(A, T).squeeze(1)
        T2 = self.ca_TM(T, M).squeeze(1)
        M2 = self.ca_MA(M, A).squeeze(1)
        
        fused = self.gate(A2, T2, M2)
        fused = self.dropout_fuse(fused)
        return self.clf(fused)

In [11]:
model = MultiModalFusion().to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.25)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

def train_one_epoch(model, loader):
    model.train()
    tot_loss, tot_correct, tot = 0, 0, 0
    for A, T, M, y in loader:
        A, T, M, y = A.to(device), T.to(device), M.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(A, T, M)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        tot_loss += loss.item()
        tot_correct += (out.argmax(1) == y).sum().item()
        tot += len(y)
    return tot_loss / len(loader), tot_correct / tot

def validate_one_epoch(model, loader):
    model.eval()
    tot_loss, tot_correct, tot = 0, 0, 0
    with torch.no_grad():
        for A, T, M, y in loader:
            A, T, M, y = A.to(device), T.to(device), M.to(device), y.to(device)
            out = model(A, T, M)
            loss = criterion(out, y)
            tot_loss += loss.item()
            tot_correct += (out.argmax(1) == y).sum().item()
            tot += len(y)
    return tot_loss / len(loader), tot_correct / tot

In [12]:
print("\n" + "="*60)
print("🎯 TRAINING MULTIMODAL FUSION MODEL")
print("="*60)

EPOCHS = 30
best_acc = 0
best_state = None
wait = 0
patience = 7

for e in range(EPOCHS):
    tr_l, tr_a = train_one_epoch(model, train_loader)
    va_l, va_a = validate_one_epoch(model, val_loader)
    scheduler.step()
    
    print(f"Epoch {e+1:2d}/{EPOCHS} | Train Loss={tr_l:.4f} Acc={tr_a:.4f} | Val Loss={va_l:.4f} Acc={va_a:.4f}")
    
    if va_a > best_acc:
        best_acc = va_a
        best_state = model.state_dict()
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("\n🛑 EARLY STOPPING")
            break

# Load best model
model.load_state_dict(best_state)


🎯 TRAINING MULTIMODAL FUSION MODEL
Epoch  1/30 | Train Loss=1.6044 Acc=0.2355 | Val Loss=1.6002 Acc=0.2541
Epoch  2/30 | Train Loss=1.6002 Acc=0.2632 | Val Loss=1.5892 Acc=0.3204
Epoch  3/30 | Train Loss=1.5883 Acc=0.2742 | Val Loss=1.5729 Acc=0.3923
Epoch  4/30 | Train Loss=1.5666 Acc=0.3338 | Val Loss=1.5439 Acc=0.3702
Epoch  5/30 | Train Loss=1.5377 Acc=0.3753 | Val Loss=1.5273 Acc=0.3646
Epoch  6/30 | Train Loss=1.5116 Acc=0.3823 | Val Loss=1.5124 Acc=0.3812
Epoch  7/30 | Train Loss=1.4928 Acc=0.4072 | Val Loss=1.4890 Acc=0.4309
Epoch  8/30 | Train Loss=1.4878 Acc=0.4335 | Val Loss=1.4836 Acc=0.4586
Epoch  9/30 | Train Loss=1.4664 Acc=0.4488 | Val Loss=1.4769 Acc=0.4641
Epoch 10/30 | Train Loss=1.4556 Acc=0.4709 | Val Loss=1.4668 Acc=0.4586
Epoch 11/30 | Train Loss=1.4424 Acc=0.4765 | Val Loss=1.4633 Acc=0.4917
Epoch 12/30 | Train Loss=1.4309 Acc=0.4751 | Val Loss=1.4638 Acc=0.4917
Epoch 13/30 | Train Loss=1.4241 Acc=0.4903 | Val Loss=1.4730 Acc=0.4641
Epoch 14/30 | Train Loss=1.4

<All keys matched successfully>

In [13]:
print("\n" + "="*60)
print("✅ FINAL EVALUATION")
print("="*60)

model.eval()

# Train metrics
train_true, train_pred = [], []
with torch.no_grad():
    for A, T, M, y in train_loader:
        A, T, M = A.to(device), T.to(device), M.to(device)
        pred = model(A, T, M).argmax(1).cpu().numpy()
        train_pred.extend(pred)
        train_true.extend(y.numpy())

# Val metrics
val_true, val_pred = [], []
with torch.no_grad():
    for A, T, M, y in val_loader:
        A, T, M = A.to(device), T.to(device), M.to(device)
        pred = model(A, T, M).argmax(1).cpu().numpy()
        val_pred.extend(pred)
        val_true.extend(y.numpy())

# Print results
print("\n📊 PERFORMANCE METRICS:")
print(f"TRAIN → Acc: {accuracy_score(train_true, train_pred):.4f} | "
      f"F1-Macro: {f1_score(train_true, train_pred, average='macro'):.4f} | "
      f"MAE: {mean_absolute_error(train_true, train_pred):.4f}")
print(f"VAL   → Acc: {accuracy_score(val_true, val_pred):.4f} | "
      f"F1-Macro: {f1_score(val_true, val_pred, average='macro'):.4f} | "
      f"MAE: {mean_absolute_error(val_true, val_pred):.4f}")

print("\n📋 Classification Report (Validation):")
labels = ["Cluster 1", "Cluster 2", "Cluster 3", "Cluster 4", "Cluster 5"]
print(classification_report(val_true, val_pred, target_names=labels))

print("\n📉 Confusion Matrix:")
cm = confusion_matrix(val_true, val_pred)
print(cm)


✅ FINAL EVALUATION

📊 PERFORMANCE METRICS:
TRAIN → Acc: 0.5845 | F1-Macro: 0.5541 | MAE: 0.8532
VAL   → Acc: 0.4751 | F1-Macro: 0.4279 | MAE: 1.0663

📋 Classification Report (Validation):
              precision    recall  f1-score   support

   Cluster 1       0.27      0.12      0.16        34
   Cluster 2       0.62      0.24      0.35        33
   Cluster 3       0.51      0.81      0.62        43
   Cluster 4       0.46      0.45      0.45        38
   Cluster 5       0.47      0.67      0.55        33

    accuracy                           0.48       181
   macro avg       0.46      0.46      0.43       181
weighted avg       0.46      0.48      0.44       181


📉 Confusion Matrix:
[[ 4  5  6  5 14]
 [ 4  8  9  7  5]
 [ 1  0 35  4  3]
 [ 3  0 15 17  3]
 [ 3  0  4  4 22]]


In [14]:
torch.save({
    'model_state': best_state,
    'midi_encoder_state': midi_encoder.state_dict(),
    'performance': {
        'train_acc': accuracy_score(train_true, train_pred),
        'val_acc': accuracy_score(val_true, val_pred),
        'val_f1': f1_score(val_true, val_pred, average='macro')
    }
}, "/kaggle/working/multimodal_emotion_model.pth")

print("\n💾 Model saved to: /kaggle/working/multimodal_emotion_model.pth")
print("\n" + "="*60)
print("🎉 TRAINING COMPLETE!")
print("="*60)


💾 Model saved to: /kaggle/working/multimodal_emotion_model.pth

🎉 TRAINING COMPLETE!
